<a href="https://colab.research.google.com/github/sitikanthaprusty2904/Pytorch_sp/blob/main/ANN_fmnist_GPU_version_60000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt

#ok bro

In [96]:
# set random seed for reproductibility
torch.manual_seed(42)


In [97]:
# check GPU availibility
device='cpu'
if torch.cuda.is_available():
  device=torch.device('cuda')
print('device: ',device)

device:  cuda


In [98]:
df=pd.read_csv('fashion-mnist_train.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
df.shape

(60000, 785)

In [100]:
# train test split
X=df.iloc[:,1:]
y=df.iloc[:,0]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [101]:
# scaling as max pixel is 255
X_train=X_train/255.0
X_test=X_test/255.0

In [102]:
class Customdataset(Dataset):
  def __init__(self,x,y):
    self.x=torch.tensor(x.values,dtype=torch.float32)
    self.y=torch.tensor(y.values,dtype=torch.long)
  def __len__(self):
    return len(self.x)
  def __getitem__(self, index) :
    return self.x[index],self.y[index]


In [103]:
# create train dataset object
train_dataset=Customdataset(X_train,y_train)
train_dataloader=DataLoader(train_dataset,batch_size=480,shuffle=True,pin_memory=True)

#create test data set object
test_dataset=Customdataset(X_test,y_test)
test_dataloader=DataLoader(test_dataset,batch_size=480,shuffle=False,pin_memory=True)

In [104]:
# nn module class
class MyNN(nn.Module):
  def __init__(self,x_num):
    super().__init__()
    self.model=nn.Sequential(
        nn.Linear(x_num,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),

        nn.Linear(64,10)

        )
  def forward(self,x):
    return self.model(x)


In [105]:
# set lr and epoch
lr=0.01
epoch=100

In [106]:
# initiating the model
model=MyNN(X_train.shape[1])
model=model.to(device)

#loss
loss=nn.CrossEntropyLoss()

#optimizer
optimizer=optim.Adam(model.parameters(),lr=lr)

# len(train_dataloader)

In [107]:
# training loop

for epoch in range (epoch):
  tl=0
  for batch_featurs,batch_labels in train_dataloader:
    #moving to gpu
    batch_featurs=batch_featurs.to(device)
    batch_labels=batch_labels.to(device)
    # forward pass
    output=model(batch_featurs)
    #loss
    loss_value=loss(output,batch_labels)
    #backward
    optimizer.zero_grad()
    loss_value.backward()
    #update grad
    optimizer.step()
    tl+=loss_value.item()
  print(f"epoch {epoch+1} loss {tl/len(train_dataloader)}")


epoch 1 loss 0.650242635011673
epoch 2 loss 0.40627713590860365
epoch 3 loss 0.36285489588975905
epoch 4 loss 0.34132409244775774
epoch 5 loss 0.32032953187823293
epoch 6 loss 0.3054966226220131
epoch 7 loss 0.29567243993282316
epoch 8 loss 0.28598768889904025
epoch 9 loss 0.2834609825909138
epoch 10 loss 0.2685775351524353
epoch 11 loss 0.2563161215186119
epoch 12 loss 0.2589931057393551
epoch 13 loss 0.25036831945180893
epoch 14 loss 0.24507944241166116
epoch 15 loss 0.23346840143203734
epoch 16 loss 0.22884146869182587
epoch 17 loss 0.23223177298903466
epoch 18 loss 0.23370563969016075
epoch 19 loss 0.2208857122063637
epoch 20 loss 0.21808163449168205
epoch 21 loss 0.2072797791659832
epoch 22 loss 0.20827357061207294
epoch 23 loss 0.20563485831022263
epoch 24 loss 0.20231017597019674
epoch 25 loss 0.20500128611922264
epoch 26 loss 0.21066302701830864
epoch 27 loss 0.20136440232396124
epoch 28 loss 0.19146577529609204
epoch 29 loss 0.18893444940447807
epoch 30 loss 0.1922743745148181

In [108]:
# set model for evaluation
model.eval()  # mode change for evaluation


MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [109]:
# evaluation code
total=0
correct=0

with torch.no_grad():
  for batch_featurs,batch_labels in test_dataloader:
    #moving to gpu
    batch_featurs=batch_featurs.to(device)
    batch_labels=batch_labels.to(device)

    output=model(batch_featurs)
    _,predicted=torch.max(output.data,1)
    total+= batch_labels.shape[0]
    correct+= (predicted==batch_labels).sum().item()
print(f"Accuracy: {correct/total}")


Accuracy: 0.8745833333333334


In [110]:
len(test_dataloader)

25